At some point during the notebook, you will have to leve this notebook and use Cellpose, and then come back to this notebook.

To learn how to use cell pose, check:

https://www.youtube.com/watch?v=5qANHWoubZU   or  https://www.youtube.com/watch?v=wB7XYh4QRiI   (short version)


Warning: in the first notebook "Read_files_create_batch.ipynb" we created and stored files in particular forders
 {batch_name}/{condition}/{channel}. Depending on what do you want to segment, go to each corresponding folder and run cellpose and retrain it.
 To run cellpose, just open a command line, open the conda environment (base_env) and just run ">>Cellpose". Then drag and drop a file from the desired folder into cellpose.
 For instance, to train a model to get the cell bodies, if your experiment has cellmask on top of the DAPI channel, you can segment with cell pose using the DAPI folder (and the right diamenter parameter). Otherwise, you may try to use the FISH data. Cellpose will use the images present in the folder from where you dropped the first image, and it will create a folder called models. That is why it is a good practice to separate the images by channels. 

In Cellpose, read the instructions to retrain a model in: Models / Training instructions. Do not forget, once you resegment to press Ctrl  + S to save the masks.

Warning: if you use a same set of images to train two different models (for instance when using cellmask with the dapi, you will train a model for the nuclei and a model for the cellbodies).
The masks of the nuclei saved by cellpose in the images folder might get in conflict with the new masks. When you trained the model, delete those masks by hand.

Very important: when you train a model on the cell body or on the nuclei, note down the diameter, this parameter will be useful in this notebook, and stored at the end of it.
Also, when training two different models on a same dataset, both models will be saved in /models. One strategy in order to do not confuse them is to rename them with a more explicit name
such as date_cyto3_nucleus to do not confuse them. The model is the file without extension. In this notebook, we will store the model name in the .json file, however you must be able to recognize it to use it.
 

   Jacques Bourg @ Florian Muller lab. Institut Pasteur.


<img src="./pipeline.png" alt="pipeline" width="1200" height="477">


In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
import skimage.io as io
import napari

import ipywidgets as widgets
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
sys.path.append(str(base_dir / "segmentation"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.file_handling import FileProcessor as Fp
from utils.widgets import IntegerInputWidget as Iiw
from segmentation.refine_seg import Segmentation

fp  = Fp()
tk  = Track()
sg  = Segmentation()

In [ ]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

In [ ]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0]
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

In [ ]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

In [ ]:
n2  = np.where(np.array(modalities) == dropdown2.value)[0][0] 
modality = modalities[n2]; print(modality)

#### Choose the channel which was used to segment the structure of interest (nuclei or cell)

In [ ]:
channel   = constants['CHANNELS']
dropdown3 = widgets.Dropdown(options=channel, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown3)

In [ ]:
n3   = np.where(np.array(channel) == dropdown3.value)[0][0]
chan = channel[n3]; print(chan)

In [ ]:
structs   = constants['STRUCTURES']
dropdown4 = widgets.Dropdown(options=structs, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown4)

In [ ]:
n4   = np.where(np.array(structs) == dropdown4.value)[0][0]
struc = structs[n4]; print(struc)

In [ ]:
# choose  model
folder_models          = fp.select_folder(initialdir = str(Path(f'../Analysis/{batch_name}/{modality}/{chan}/{struc}/train_2D/')), title="Select the path to the cell pose models folder (ex  .../FISH/.../train2D/models) ")     
batch_models           = os.listdir(folder_models)
batch_models_filtered  = [el for el in batch_models if Path(el).suffix == '']
dropdown_model         = widgets.Dropdown(options=batch_models_filtered, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown_model)

In [ ]:
n4 = np.where((np.array(batch_models) == dropdown_model.value))[0][0]
model= batch_models[n4]; print(model)

### Segmentation of nuclei

##### Define the diameter of the nuclei in pixels (medium diameter in pixels computed during training). It is better if you noted it, othrwise for HeLa cells d nuc ≈ 150 pix

In [ ]:
iw = Iiw(title='Diameter (px)')
iw.display()

In [ ]:
diam_deblur = iw.input.value; print('Diameter (pixels):', diam_deblur)

In [ ]:
batch_mod_chan_mip = constants[f'BATCH_{modality}_{chan}_{struc}_MIP']
mask_folder_path   = Path(f"../Analysis/{batch_name}/{modality}/{chan}/{struc}/masks")
if not mask_folder_path.exists():
    mask_folder_path.mkdir(parents=True)

GPU_CP               = True
pretrained_model_path= str(Path(folder_models) / model)

mask_list_path       = mask_folder_path / Path(f"{batch_name}_masks_{modality}_{chan}_{struc}.npy")
mask_contours_path   = mask_folder_path / Path(f"{batch_name}_contours_{modality}_{chan}_{struc}.npy")

viewer1         = napari.Viewer(title="Cell pose segmentation")
dict_mask_cell  = {}
dict_contours   = {}

counter = 0
for ind, file in enumerate(batch_mod_chan_mip):
    file        = Path(file)
    im_cell     = io.imread(file)

    cell_db     = sg.deblur_cellpose(im_cell, diameter=diam_deblur, gpu=GPU_CP)
    masks_cell  = sg.segment_with_custom_model(cell_db, pretrained_model_path, gpu=GPU_CP)
    countours   = sg.find_all_contours(masks_cell)
    
    value_max   = np.percentile(im_cell, 99)
    viewer1.add_image(im_cell, rgb=False, name=f" {str(file.stem)}", contrast_limits=(0, value_max))
    viewer1.add_shapes(countours, name=f"Contours {str(file.stem)}", shape_type='polygon',
                                                edge_color='red', face_color='transparent', opacity=1)
    
    base_name   = '_'.join(Path(file).stem.split('_')[:-3])
    dict_mask_cell[base_name] = masks_cell
    dict_contours[base_name]  = countours

    if counter != 0:
        viewer1.layers[f" {str(file.stem)}"].visible         = False
        viewer1.layers[f"Contours {str(file.stem)}"].visible = False 
    counter += 1
      
fp.save_masks_distributed_files(mask_list_path, dict_mask_cell)
np.save(mask_contours_path, dict_contours)

##### turn important lower case variables into uppercase for tracking, create those variables names with this content: useful for experiment tracking

In [ ]:
exec(f"MASK_{modality}_{chan}_{struc}_CONTOURS_PATH = mask_contours_path", globals())
exec(f"MASK_{modality}_{chan}_{struc}_LIST_PATH = mask_list_path", globals())
exec(f"DIAMETER_CELLPOSE_{modality}_{chan}_{struc} = diam_deblur", globals())
exec(f"MODEL_PATH_{modality}_{chan}_{struc} = pretrained_model_path", globals())
exec(f"MODEL_NAME_{modality}_{chan}_{struc} = '{model}' ", globals())

Save parameters

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))